To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News


Unsloth's [Docker image](https://hub.docker.com/r/unsloth/unsloth) is here! Start training with no setup & environment issues. [Read our Guide](https://docs.unsloth.ai/new/how-to-train-llms-with-unsloth-and-docker).

[gpt-oss RL](https://docs.unsloth.ai/new/gpt-oss-reinforcement-learning) is now supported with the fastest inference & lowest VRAM. Try our [new notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/gpt-oss-(20B)-GRPO.ipynb) which creates kernels!

Introducing [Vision](https://docs.unsloth.ai/new/vision-reinforcement-learning-vlm-rl) and [Standby](https://docs.unsloth.ai/basics/memory-efficient-rl) for RL! Train Qwen, Gemma etc. VLMs with GSPO - even faster with less VRAM.

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [23]:
!pip install unsloth
# !pip install transformers==4.55.4
# !pip install --no-deps trl==0.22.2


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [24]:
# %%capture
# # These are mamba kernels and we must have these for faster training
# !pip install --no-build-isolation mamba_ssm==2.2.5
# !pip install --no-build-isolation causal_conv1d==1.5.2

### Unsloth

In [25]:
from unsloth import FastLanguageModel
import torch

fourbit_models = [
    "unsloth/granite-4.0-micro",
    "unsloth/granite-4.0-h-micro",
    "unsloth/granite-4.0-h-tiny",
    "unsloth/granite-4.0-h-small",

    # Base pretrained Granite 4 models
    "unsloth/granite-4.0-micro-base",
    "unsloth/granite-4.0-h-micro-base",
    "unsloth/granite-4.0-h-tiny-base",
    "unsloth/granite-4.0-h-small-base",

    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/Phi-4",
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/orpheus-3b-0.1-ft-unsloth-bnb-4bit" # [NEW] We support TTS models!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/granite-4.0-micro",
    max_seq_length = 2048,   # Choose any for long context!
    load_in_4bit = True,    # 4 bit quantization to reduce memory
    load_in_8bit = False,    # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
)

==((====))==  Unsloth 2025.10.1: Fast Granitemoehybrid patching. Transformers: 4.56.2.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.179 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu129. CUDA: 9.0. CUDA Toolkit: 12.9. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update a small amount of parameters!

In [26]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "shared_mlp.input_linear", "shared_mlp.output_linear"],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Making `model.base_model.model.model` require gradients


<a name="Data"></a>
### Data Prep
#### 📄 Using Google Sheets as Training Data
Our goal is to create a customer support bot that proactively helps and solves issues.

We’re storing examples in a Google Sheet with two columns:

- **Snippet**: A short customer support interaction
- **Recommendation**: A suggestion for how the agent should respond

This keeps things simple and collaborative. Anyone can edit the sheet, no database setup required.  
<br>

---
<br>

#### 🔍 Why This Format?

This setup works well for tasks like:

- `Input snippet → Suggested reply`
- `Prompt → Rewrite`
- `Bug report → Diagnosis`
- `Text → Label or Category`

Just collect examples in a spreadsheet, and you’ve got usable training data.  
<br>

---
<br>

#### ✅ What You'll Learn

We’ll show how to:

1. Load the Google Sheet into your notebook
2. Format it into a dataset
3. Use it to train or prompt an LLM


The chat template for granite-4 look like this:
```
<|start_of_role|>system<|end_of_role|>Knowledge Cutoff Date: April 2024.
Today's Date: June 24, 2025.
You are Granite, developed by IBM. You are a helpful AI assistant.<|end_of_text|>

<|start_of_role|>user<|end_of_role|>How do astronomers determine the original wavelength of light emitted by a celestial body at rest, which is necessary for measuring its speed using the Doppler effect?<|end_of_text|>

<|start_of_role|>assistant<|end_of_role|>Astronomers make use of the unique spectral fingerprints of elements found in stars...<|end_of_text|>
```

In [27]:
import pandas as pd
import numpy as np
import re
from datasets import Dataset

# Enhanced text cleaning function - extracts key features AND keeps full text
def clean_text_enhanced(text):
    if pd.isnull(text):
        return ""
    
    # Convert to string and clean basic issues
    text = str(text).strip()
    
    # Extract ALL structured information (not just top 3)
    item_name = re.search(r"Item Name:\s*(.*?)(?=\n|$)", text, re.IGNORECASE)
    brand = re.search(r"Brand:\s*(.*?)(?=\n|$)", text, re.IGNORECASE)
    color = re.search(r"Color:\s*(.*?)(?=\n|$)", text, re.IGNORECASE)
    size = re.search(r"Size:\s*(.*?)(?=\n|$)", text, re.IGNORECASE)
    material = re.search(r"Material:\s*(.*?)(?=\n|$)", text, re.IGNORECASE)
    model = re.search(r"Model:\s*(.*?)(?=\n|$)", text, re.IGNORECASE)
    
    # Extract bullet points (all of them)
    bp1 = re.search(r"Bullet Point\s*1:\s*(.*?)(?=\n|$)", text, re.IGNORECASE)
    bp2 = re.search(r"Bullet Point\s*2:\s*(.*?)(?=\n|$)", text, re.IGNORECASE)
    bp3 = re.search(r"Bullet Point\s*3:\s*(.*?)(?=\n|$)", text, re.IGNORECASE)
    bp4 = re.search(r"Bullet Point\s*4:\s*(.*?)(?=\n|$)", text, re.IGNORECASE)
    bp5 = re.search(r"Bullet Point\s*5:\s*(.*?)(?=\n|$)", text, re.IGNORECASE)
    
    # Extract value and unit
    value = re.search(r"Value:\s*([\d.,]+)", text, re.IGNORECASE)
    unit = re.search(r"Unit:\s*([A-Za-z]+)", text, re.IGNORECASE)
    
    # Extract description if present
    description = re.search(r"Description:\s*(.*?)(?=\n|$)", text, re.IGNORECASE)
    
    # Build structured output with KEY features first, then append everything else
    structured_parts = []
    
    # Top priority features (Item Name, Value, Unit)
    if item_name:
        structured_parts.append(f"Item: {item_name.group(1).strip()}")
    if value and unit:
        structured_parts.append(f"Quantity: {value.group(1).strip()} {unit.group(1).strip()}")
    elif value:
        structured_parts.append(f"Value: {value.group(1).strip()}")
    
    # Additional important features
    if brand:
        structured_parts.append(f"Brand: {brand.group(1).strip()}")
    if color:
        structured_parts.append(f"Color: {color.group(1).strip()}")
    if size:
        structured_parts.append(f"Size: {size.group(1).strip()}")
    if material:
        structured_parts.append(f"Material: {material.group(1).strip()}")
    if model:
        structured_parts.append(f"Model: {model.group(1).strip()}")
    
    # All bullet points
    if bp1:
        structured_parts.append(f"Feature 1: {bp1.group(1).strip()}")
    if bp2:
        structured_parts.append(f"Feature 2: {bp2.group(1).strip()}")
    if bp3:
        structured_parts.append(f"Feature 3: {bp3.group(1).strip()}")
    if bp4:
        structured_parts.append(f"Feature 4: {bp4.group(1).strip()}")
    if bp5:
        structured_parts.append(f"Feature 5: {bp5.group(1).strip()}")
    
    if description:
        structured_parts.append(f"Description: {description.group(1).strip()}")
    
    # Join structured parts
    cleaned_text = ". ".join(structured_parts)
    
    # IMPORTANT: Append the FULL original text (cleaned) so nothing is lost
    # This ensures ALL information is available to the model
    full_text_cleaned = text.lower()
    full_text_cleaned = re.sub(r'[^\w\s.,:\-]', ' ', full_text_cleaned)
    full_text_cleaned = re.sub(r'\s+', ' ', full_text_cleaned)
    full_text_cleaned = full_text_cleaned.strip()
    
    # Combine: structured features first, then full text for additional context
    if cleaned_text and full_text_cleaned:
        final_text = f"{cleaned_text}. Full Details: {full_text_cleaned}"
    elif cleaned_text:
        final_text = cleaned_text
    else:
        final_text = full_text_cleaned
    
    return final_text

print("Loading training data from dataset/train.csv...")
train_df = pd.read_csv('/root/train.csv', encoding='latin1')

print(f"Original data shape: {train_df.shape}")
print(f"Columns: {train_df.columns.tolist()}")

# Apply text cleaning
print("\nApplying enhanced text cleaning...")
train_df['catalog_content'] = train_df['catalog_content'].apply(clean_text_enhanced)

# Filter out empty or very short text
train_df['text_length'] = train_df['catalog_content'].str.len()
train_df = train_df[train_df['text_length'] > 10].copy()

print(f"Data shape after cleaning: {train_df.shape}")
print(f"\nPrice statistics:")
print(train_df['price'].describe())

# Convert to HuggingFace Dataset format
dataset = Dataset.from_pandas(train_df[['catalog_content', 'price']])

print(f"\n✅ Dataset loaded: {len(dataset)} samples")

Loading training data from dataset/train.csv...
Original data shape: (75000, 4)
Columns: ['sample_id', 'catalog_content', 'image_link', 'price']

Applying enhanced text cleaning...
Data shape after cleaning: (75000, 5)

Price statistics:
count    75000.000000
mean        23.647654
std         33.376932
min          0.130000
25%          6.795000
50%         14.000000
75%         28.625000
max       2796.000000
Name: price, dtype: float64

✅ Dataset loaded: 75000 samples


We've just loaded the Google Sheet as a csv style Dataset, but we still need to format it into conversational style like below and then apply the chat template.

```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

We'll use a helper function `formatting_prompts_func` to do both!

In [28]:
tokenizer.chat_template = "{{ bos_token }}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ message['content'] + eos_token}}{% else %}{{ raise_exception('Only user and assistant roles are supported!') }}{% endif %}{% endfor %}"

def formatting_prompts_func(examples):
    catalog_texts = examples['catalog_content']
    prices = examples['price']
    
    messages = [
        [{"role": "user", "content": f"Predict the price for this product: {catalog_text}"},
         {"role": "assistant", "content": f"The predicted price is ${price:.2f}"}] 
        for catalog_text, price in zip(catalog_texts, prices)
    ]
    
    # This will now work correctly
    texts = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) 
             for message in messages]
    
    return {"text": texts}

print("Formatting dataset with chat template...")
dataset = dataset.map(formatting_prompts_func, batched=True)
print(f"✅ Dataset formatted: {len(dataset)} samples")

Formatting dataset with chat template...


Map:   0%|          | 0/75000 [00:00<?, ? examples/s]

✅ Dataset formatted: 75000 samples


We now look at the raw input data before formatting.

In [29]:
# Show raw catalog content before formatting
print("Sample catalog content:")
print(dataset[5]["catalog_content"][:500])  # Show first 500 chars

Sample catalog content:
Item: Member's Mark Member's Mark, Basil, 6.25 oz. Quantity: 6.25 ounce. Feature 1: Green Herb, Italian Staple, Great mixed with Oregano. Feature 2: Large Size, Chef Bottle. Feature 3: Packed in the USA. Full Details: item name: member s mark member s mark, basil, 6.25 oz bullet point 1: green herb, italian staple, great mixed with oregano bullet point 2: large size, chef bottle bullet point 3: packed in the usa value: 6.25 unit: ounce


In [30]:
# Show the corresponding price
print("Sample price:")
print(f"${dataset[5]['price']:.2f}")

Sample price:
$18.50


And we see how the chat template transformed these conversations.

In [31]:
dataset[5]["text"]

"<|end_of_text|>[INST] Predict the price for this product: Item: Member's Mark Member's Mark, Basil, 6.25 oz. Quantity: 6.25 ounce. Feature 1: Green Herb, Italian Staple, Great mixed with Oregano. Feature 2: Large Size, Chef Bottle. Feature 3: Packed in the USA. Full Details: item name: member s mark member s mark, basil, 6.25 oz bullet point 1: green herb, italian staple, great mixed with oregano bullet point 2: large size, chef bottle bullet point 3: packed in the usa value: 6.25 unit: ounce [/INST]The predicted price is $18.50<|end_of_text|>"

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [32]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 35,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=97):   0%|          | 0/75000 [00:00<?, ? examples/s]

Detected kernel version 4.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [33]:
# from unsloth.chat_templates import train_on_responses_only
# trainer = train_on_responses_only(
#     trainer,
#     instruction_part = "<|start_of_role|>user<|end_of_role|>",
#     response_part = "<|start_of_role|>assistant<|end_of_role|>",
# )

Let's verify masking the instruction part is done! Let's print the 100th row again.

In [34]:
# Verify the full formatted text (input_ids)
if len(trainer.train_dataset) > 100:
    print("Full formatted example:")
    print(tokenizer.decode(trainer.train_dataset[100]["input_ids"]))
else:
    print(f"Dataset only has {len(trainer.train_dataset)} samples. Showing first sample:")
    print(tokenizer.decode(trainer.train_dataset[0]["input_ids"]))

Full formatted example:
<|end_of_text|>[INST] Predict the price for this product: Item: Amazon Grocery, Lemonade Drink Mix, 10 packets, 1.4 Oz (Previously Happy Belly, Packaging May Vary). Quantity: 1.4 Ounce. Feature 1: 10 packets of Lemonade Drink Mix. Feature 2: Some of your favorite Happy Belly products are now part of the Amazon Grocery brand! Although packaging may vary during the transition, the ingredients and product remain the same. Thank you for your continued trust in our brands. Feature 3: Sugar Free, Low Sodium. Feature 4: 10 calories per serving. Feature 5: Amazon Grocery has all the favorites you love for less. Youâll find everything you need for great-tasting meals in one shopping trip. Description: 10 packets of Lemonade Drink Mix. Full Details: item name: amazon grocery, lemonade drink mix, 10 packets, 1.4 oz previously happy belly, packaging may vary bullet point 1: 10 packets of lemonade drink mix bullet point 2: some of your favorite happy belly products are now

Now let's print the masked out example - you should see only the answer is present:

In [35]:
# Now let's print the masked out example - you should see only the assistant response
if len(trainer.train_dataset) > 100:
    sample_idx = 100
else:
    sample_idx = 0

if "labels" in trainer.train_dataset[sample_idx]:
    masked_labels = [tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[sample_idx]["labels"]]
    decoded = tokenizer.decode(masked_labels)
    if tokenizer.pad_token:
        decoded = decoded.replace(tokenizer.pad_token, " ")
    print("Masked output (only assistant response should be visible):")
    print(decoded)
else:
    print("Labels field not found. The masking will be applied during training.")

Labels field not found. The masking will be applied during training.


In [36]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA H100 80GB HBM3. Max memory = 79.179 GB.
5.959 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

```
Notice you might have to wait ~10 minutes for the Mamba kernels to compile! Please be patient!
```

In [37]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 75,000 | Num Epochs = 2 | Total steps = 962
O^O/ \_/ \    Batch size per device = 39 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (39 x 4 x 1) = 156
 "-____-"     Trainable parameters = 58,982,400 of 3,461,818,880 (1.70% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.813000


KeyboardInterrupt: 

<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! We'll use some example snippets not contained in our training data to get a sense of what was learned.

In [38]:
import pandas as pd

print("Loading test data from dataset/test.csv...")
test_df = pd.read_csv('/root/test.csv', encoding='latin1')

print(f"Test data shape: {test_df.shape}")

# Apply same text cleaning
test_df['catalog_content'] = test_df['catalog_content'].apply(clean_text_enhanced)

# Prepare for inference
FastLanguageModel.for_inference(model)

# Predict prices for all test samples
all_predictions = []

print("\nGenerating predictions...")
from transformers import TextStreamer

for idx in range(min(5, len(test_df))):  # Show first 5 as examples
    catalog_text = test_df.iloc[idx]['catalog_content']
    sample_id = test_df.iloc[idx]['sample_id']
    
    messages = [
        {"role": "user", "content": f"Predict the price for this product: {catalog_text}"},
    ]
    
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        padding=True,
        return_tensors="pt",
        return_dict=True,
    ).to("cuda")
    
    print(f"\n--- Sample {sample_id} ---")
    text_streamer = TextStreamer(tokenizer, skip_prompt=True)
    
    outputs = model.generate(**inputs,
                             streamer=text_streamer,
                             max_new_tokens=64,
                             use_cache=True,
                             do_sample=False,
                             temperature=0.1)
    
    # Extract predicted price from output
    # You'll need to parse the generated text to extract the price value

print("\n✅ Sample predictions complete!")
print("\nNote: For full test set predictions, loop through all samples and save to submission.csv")

Loading test data from dataset/test.csv...
Test data shape: (75000, 3)

Generating predictions...

--- Sample 100179 ---
<|end_of_text|>

--- Sample 245611 ---
"

Based on the information provided, here is my prediction for the price of the "Natural MILK TEA Flavoring Extract by HALO PANTRY (2oz bottle)" product:

Predicted Price: $12.99

Reasoning:
- The product is described as a "premium line" of Asian-inspired flavor

--- Sample 146263 ---
<|end_of_text|>

--- Sample 95658 ---
<|end_of_text|>

--- Sample 36806 ---
<|end_of_text|>

✅ Sample predictions complete!

Note: For full test set predictions, loop through all samples and save to submission.csv


In [39]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# messages = [
#     {"role": "user", "content": "tex"},
# ]
# inputs = tokenizer.apply_chat_template(
#     messages,
#     tokenize = True,
#     add_generation_prompt = True, # Must add for generation
#     padding = True,
#     return_tensors = "pt",
#     return_dict=True,
# ).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)

# _ = model.generate(**inputs,
#                    streamer = text_streamer,
#                    max_new_tokens = 256, # Reduced for faster inference
#                    use_cache = True,
#                    # Greedy decoding for more deterministic responses
#                    do_sample=False,
#                    temperature = 0.5, top_p = 0.9, top_k = 40,
# )

In [ ]:
# Create full submission file
import re
import pandas as pd
from tqdm.notebook import tqdm  # Import tqdm

print("Creating submission file for all test samples...")

test_df = pd.read_csv('/root/train.csv', encoding='latin1')
test_df['catalog_content'] = test_df['catalog_content'].apply(clean_text_enhanced)

FastLanguageModel.for_inference(model)

all_predictions = []

# Wrap the range object with tqdm to create a progress bar
for idx in tqdm(range(len(test_df)), desc="Generating Predictions"):
    catalog_text = test_df.iloc[idx]['catalog_content']
    
    messages = [
        {"role": "user", "content": f"Predict the price for this product: {catalog_text}"},
    ]
    
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        padding=True,
        return_tensors="pt",
        return_dict=True,
    ).to("cuda")
    
    outputs = model.generate(**inputs,
                             max_new_tokens=64,
                             use_cache=True,
                             do_sample=False,
                             temperature=0.1)
    
    # Decode output
    predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract price from text (look for patterns like $XX.XX or price is XX.XX)
    price_match = re.search(r'\$(\d+\.?\d*)|price is (\d+\.?\d*)', predicted_text, re.IGNORECASE)
    
    if price_match:
        price = float(price_match.group(1) or price_match.group(2))
    else:
        # Fallback to a default reasonable price
        price = 50.0
    
    all_predictions.append(price)
    
    # This manual print statement is no longer needed
    # if (idx + 1) % 100 == 0:
    #     print(f"Processed {idx + 1}/{len(test_df)} samples")

# Create submission DataFrame
submission = pd.DataFrame({
    'sample_id': test_df['sample_id'],
    'price': all_predictions
})

# Save submission
submission.to_csv('submission_granite.csv', index=False)

print(f"\n✅ Submission saved to submission_granite.csv")
print(f"Shape: {submission.shape}")
print(f"\nPrice statistics:")
print(submission['price'].describe())

Creating submission file for all test samples...


Generating Predictions:   0%|          | 0/75000 [00:00<?, ?it/s]

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
# model.save_pretrained("lora_model")  # Local saving
# tokenizer.save_pretrained("lora_model")
# # model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# # tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
# if False:
#     from unsloth import FastLanguageModel
#     model, tokenizer = FastLanguageModel.from_pretrained(
#         model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
#         max_seq_length = 2048,
#         load_in_4bit = True,
#     )

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)

model.save_pretrained("model")
    tokenizer.save_pretrained("model")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
# Save to 8bit Q8_0
if False:
    model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
